In [17]:
import pandas as pd
import xarray as xr
import numpy as np
import copy
import math
import random
import warnings
import matplotlib.pyplot as plt
import os

from deap import base
from deap import creator
from deap import tools

import calliope
from calliope.exceptions import ModelWarning

calliope.set_log_verbosity(verbosity='critical', include_solver_output=False, capture_warnings=False)

# Suppress the specific ModelWarning from Calliope
warnings.filterwarnings("ignore", category=ModelWarning)


In [18]:
#input variables
NUM_SUBPOPS = 2 # number of populations
SUBPOP_SIZE = 4 #size of populations. Must be even due to crossover
GENERATIONS = 100 #amount of generations

INDCROSS = 0.5 #chance for value of individual to be crossed over
INDMUT = 0.3 #chance for individual to mutate
ETAV = 5 #small value (1 or smaller) creates different values from parents, high value (20 or higher) creates resembling values


# changing values/resolution after n amount of generations
value_change_1 = 30  
value_change_2 = 70 

# change to other resolution after n amount of generations
resolution_change_1 = 4
resolution_change_2 = 90

# slack value
max_slack = 0.2

#unmet demand penalty. Must be changed when applied to a large model
unmetdemandvariable = 1

In [19]:
#create the model with a resolution of 14 days
model = calliope.Model("C:/Users/Jacob/Desktop/PythonProjects/GAMGA-Calliope v3.9/GAMGA_model/model_14D.yaml")
model.run()

df_total_cost = model.results.cost.to_series().dropna()
total_cost_optimal = df_total_cost.loc[~df_total_cost.index.map(str).str.contains('co2_emissions')].sum()

print(total_cost_optimal)

energy_cap_df = model.results.energy_cap.to_pandas()
filtered_energy_cap_df = energy_cap_df[~energy_cap_df.index.str.contains("demand|transmission")]

print(filtered_energy_cap_df)



c:\Users\Jacob\miniconda3\envs\calliope\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.



29888665.943493538
loc_techs
region1::battery1       0.001234
region1::ccgt        5000.000000
region1::solarPV1       0.000105
region2::battery2    2932.587565
region2::solarPV2    9910.914455
dtype: float64


In [20]:
#create the model with a resolution of 6 hours
model_FULL = calliope.Model('C:/Users/Jacob/Desktop/PythonProjects/GAMGA-Calliope v3.9/GAMGA_model/model_FULL.yaml')
model_FULL.run()

df_total_cost_FULL = model_FULL.results.cost.to_series().dropna()
total_cost_optimal_FULL = df_total_cost_FULL.loc[~df_total_cost_FULL.index.map(str).str.contains('co2_emissions')].sum()

print(total_cost_optimal_FULL)

energy_cap_df_FULL = model_FULL.results.energy_cap.to_pandas()
energycap_FULL = energy_cap_df_FULL[~energy_cap_df_FULL.index.str.contains("demand|transmission")]

print(energycap_FULL)



28573559.797458697
loc_techs
region1::battery1       0.001768
region1::ccgt        5000.000000
region1::solarPV1       0.000180
region2::battery2    2466.261028
region2::solarPV2    7375.697552
dtype: float64


In [21]:
# take initial capacities from highest resolution and use them to initiate individual values.
initial_capacities_FULL = energycap_FULL.values

# if a value is lower than 1e-5, set the value to 0. This is due to everything below 1e-5 being so low it it not interesting to include
initial_capacities_FULL[initial_capacities_FULL < 1e-5] = 0

#find Max capacity to use for inf values
max_cap_value = max(initial_capacities_FULL)

# initialize the optimal value. Used in calculating the slack distance
optimal_value = total_cost_optimal_FULL

In [22]:
#Order of technologies is stored. This is to know later on which position in the capacity list corresponds to which technology
updates = [
    {'tech': tech, 'loc': loc}
    for loc_tech in filtered_energy_cap_df.index
    for loc, tech in [loc_tech.split("::")]  # Split index by '::' to separate loc and tech
]


In [23]:

# Look in the model backend at what the values are that it has used to initialize the model
input_params = model.backend.access_model_inputs()

# Access energy_cap_max and energy_cap_min for each technology
energy_cap_max = input_params['energy_cap_max']
energy_cap_min = input_params['energy_cap_min']

# Convert to DataFrame for filtering
energy_cap_max_df = energy_cap_max.to_dataframe()
energy_cap_min_df = energy_cap_min.to_dataframe()

# Filter out rows with 'demand' or 'free' in the index
energy_cap_max_filtered = energy_cap_max_df[~energy_cap_max_df.index.get_level_values('loc_techs').str.contains("demand|transmission")]
energy_cap_min_filtered = energy_cap_min_df[~energy_cap_min_df.index.get_level_values('loc_techs').str.contains("demand|transmission")]


# Create a dictionary of loc_tech to [min, max] bounds meaning that you will have the loc_tech and their corresponding min max capacity
low_up_mapping = {
    loc_tech: [
        energy_cap_min_filtered.loc[loc_tech, 'energy_cap_min'],
        energy_cap_max_filtered.loc[loc_tech, 'energy_cap_max']
    ]
    for loc_tech in energy_cap_max_filtered.index
}

# Structure the updates (previously done, which are the tech:, loc: pairs) so that it can eventually be used to put in the backend
updates_order = [f"{update['loc']}::{update['tech']}" for update in updates]

# Put it in such a structure that it can be used for the mutation function. This wants it in a list order
low_up_bound = [
    low_up_mapping[loc_tech] for loc_tech in updates_order
]

# Check for 'inf' in upper bounds and adjust if needed
for i, (low, up) in enumerate(low_up_bound):
    if up == float('inf'):
        print(f"Technology {updates_order[i]} has 'inf' as the upper bound.")
        low_up_bound[i][1] = max_cap_value * 2 #something specific for this model
    else:
        print(f"Technology {updates_order[i]} has a finite upper bound: {up}")


Technology region1::battery1 has a finite upper bound: 10000.0
Technology region1::ccgt has a finite upper bound: 5000.0
Technology region1::solarPV1 has a finite upper bound: 15000.0
Technology region2::battery2 has a finite upper bound: 10000.0
Technology region2::solarPV2 has a finite upper bound: 15000.0


In [24]:
def update_energy_cap_max_for_individual(model, updates, individual_values):

    # Ensure the length of updates matches the individual's values
    if len(updates) != len(individual_values):
        raise ValueError("Length of updates and individual values must match.")
    
    # Update the model with the individual's capacity values
    for update, new_cap in zip(updates, individual_values):
        tech = update['tech']
        loc = update['loc']
        
        # Construct the location::technology key and update the model
        loc_tech_key = f"{loc}::{tech}"
        model.backend.update_param('energy_cap_max', {loc_tech_key: new_cap})
        model.backend.update_param('energy_cap_min', {loc_tech_key: new_cap})
    
    # Run the model for this individual
    try:
        rerun_model = model.backend.rerun()  # Rerun to capture updated backend parameters

        # Calculate the total cost, excluding emission costs
        cost_op = rerun_model.results.cost.to_series().dropna()
        initial_cost = round(cost_op.loc[~cost_op.index.map(str).str.contains('co2_emissions')].sum(), 2)

        unmet = rerun_model.results.unmet_demand.to_series().dropna()
        unmet_demand = round(unmet.sum() * unmetdemandvariable, 2) #300 is the penalty for unmet demand

        total_cost = initial_cost + unmet_demand
    
    except Exception as e:
        # If solving fails, set total cost to NaN and print a warning
        total_cost = float('inf')
        print("Warning: Model could not be solved for the individual. Assigning cost as infinite.")
    
    return total_cost

def slack_feasibility(individual):
    cost = update_energy_cap_max_for_individual(model, updates, individual)
    individual.cost = cost  # Attach cost attribute to individual
    slack_distance = (cost - optimal_value) / optimal_value

    # Update feasibility condition based on the new criteria
    feasible = slack_distance <= max_slack 
    
    return feasible 

def centroidSP(subpop):
    centroids = []

    # Iterate over each subpopulation and calculate the centroid
    for sub in subpop.values():
        if not isinstance(sub, list) or not all(isinstance(individual, list) for individual in sub):
            raise TypeError("Each subpopulation must be a list of lists (individuals).")
        
        num_solutions = len(sub)  # Number of solutions in the current subpopulation
        num_variables = len(sub[0])  # Number of decision variables
        
        # Calculate the centroid for each decision variable
        centroid = [sum(solution[i] for solution in sub) / num_solutions for i in range(num_variables)]
        centroids.append(centroid)  # Append each centroid to the main list in the required format
    
    return centroids

def fitness_euc(subpop, centroids):
    distances = []
    minimal_distances = []
    fitness_SP = {}

    # Step 1: Calculate Euclidean Distances for each individual
    for q, (subpop_index, subpopulation) in enumerate(subpop.items()):
        subpopulation_distances = []
        
        for individual in subpopulation:
            individual_distances = []
            
            for p, centroid in enumerate(centroids):
                if p != q:  # Skip the centroid of the same subpopulation
                    # Calculate Euclidean distance
                    distance = math.sqrt(sum((individual[i] - centroid[i])**2 for i in range(len(individual))))
                    individual_distances.append(distance)
            
            subpopulation_distances.append(individual_distances)
        
        distances.append(subpopulation_distances)

    # Step 2: Calculate Minimal Distances
    for subpopulation_distances in distances:
        subpopulation_minimal = [min(individual_distances) for individual_distances in subpopulation_distances]
        minimal_distances.append(subpopulation_minimal)

    # Step 3: Calculate Fitness SP for each individual
    for sp_index, subpopulation in enumerate(minimal_distances, start=1):
        fitness_values = [(min_distance,) for min_distance in subpopulation]
        fitness_SP[sp_index] = fitness_values

    return fitness_SP

def fitness(subpop, centroids):
    distances = []
    minimal_distances = []
    fitness_SP = {}

    # Step 1: Calculate Distances per Variable for each individual
    for q, (subpop_index, subpopulation) in enumerate(subpop.items()):
        subpopulation_distances = []
        
        for individual in subpopulation:
            individual_variable_distances = []
            
            for p, centroid in enumerate(centroids):
                if p != q:  # Skip the centroid of the same subpopulation
                    variable_distances = [abs(individual[i] - centroid[i]) for i in range(len(individual))]
                    individual_variable_distances.append(variable_distances)
            
            subpopulation_distances.append(individual_variable_distances)
        
        distances.append(subpopulation_distances)

    # Step 2: Calculate Minimal Distances per Variable
    for subpopulation_distances in distances:
        subpopulation_minimal = []
        
        for individual_distances in subpopulation_distances:
            min_distance_per_variable = [min(distance[i] for distance in individual_distances) for i in range(len(individual_distances[0]))]
            subpopulation_minimal.append(min_distance_per_variable)
        
        minimal_distances.append(subpopulation_minimal)

    # Step 3: Calculate Fitness SP for each individual
    for sp_index, subpopulation in enumerate(minimal_distances, start=1):
        fitness_values = [(min(individual),) for individual in subpopulation]
        fitness_SP[sp_index] = fitness_values

    return fitness_SP

def custom_tournament(subpopulation, k, tournsize=2):
    selected = []
    zero_fitness_count = 0  # Counter for individuals with fitness (0,)

    while len(selected) < k:
        # Randomly select `tournsize` individuals for the tournament
        tournament = random.sample(subpopulation, tournsize)

        # Check if all individuals in the tournament have a fitness of (0,)
        if all(ind.fitness.values == (0,) for ind in tournament):
            if zero_fitness_count < 2:
                # Select the individual with the lowest cost if all fitness values are (0,)
                best = min(tournament, key=lambda ind: ind.cost)
                selected.append(best)
                zero_fitness_count += 1
            else:
                # Select a random feasible individual if we've reached the max count of (0,) fitness values
                feasible_individuals = [ind for ind in subpopulation if ind.fitness.values != (0,)]
                if feasible_individuals:
                    best = random.choice(feasible_individuals)
                    selected.append(best)
                else:
                    # If no feasible individuals are available, fallback to random selection to avoid empty selection
                    best = random.choice(subpopulation)
                    selected.append(best)
        else:
            # Select based on fitness if there are feasible individuals in the tournament
            best = max(tournament, key=lambda ind: ind.fitness.values[0])
            selected.append(best)

    return selected

# def generate_individual():
#     adjusted_individual = []
    
#     for cap, (low, up) in zip(initial_capacities_FULL, low_up_bound):
#         if cap == 0:
#             # Small chance for installation between % of upper bound
#             if random.random() < 0.01:  # 10% chance
#                 new_value = random.uniform(0.1 * up, 0.1 * up)
#             else:
#                 # No fallback value, skip adjustment
#                 new_value = 0.0  # Keeps as zero or explicitly sets to 0
#         else:
#             # Adjust by a random value between -0.1 and 0.1 of the current value
#             adjustment = random.uniform(-0.1, 0.11)
#             new_value = cap * (1 + adjustment)
        
#         # Ensure the new value is within the lower and upper bounds
#         new_value = max(low, min(up, new_value))
#         adjusted_individual.append(new_value)
    
#     return adjusted_individual

def generate_individual():
    # Generate a new individual with capacities within the defined bounds
    adjusted_individual = [
        max(low, min(up, cap + random.choice([-1, 0, 1])))
        for cap, (low, up) in zip(initial_capacities_FULL, low_up_bound)]
    return adjusted_individual



In [25]:
creator.create("FitnessMaxDist", base.Fitness, weights=(1.0,))  # Fitness to maximize distinctiveness
creator.create("IndividualSP", list, fitness=creator.FitnessMaxDist, cost=0)  # Individual structure in DEAP

# DEAP toolbox setup
toolbox = base.Toolbox()

# Register the individual and subpopulation initializers
toolbox.register("individualSP", tools.initIterate, creator.IndividualSP, generate_individual)
toolbox.register("subpopulationSP", tools.initRepeat, list, toolbox.individualSP)

#register the operators
toolbox.register("mate", tools.cxUniform)
toolbox.register("elitism", tools.selBest, fit_attr="fitness.values")
toolbox.register("tournament", custom_tournament)
toolbox.register("mutbound", tools.mutPolynomialBounded)

# Generate subpopulations with multiple individuals
subpops_unaltered = [toolbox.subpopulationSP(n=SUBPOP_SIZE) for _ in range(NUM_SUBPOPS)]

subpops_SP = {}

for p in range(NUM_SUBPOPS):
    subpops_SP[p+1] = subpops_unaltered[p]

print(subpops_SP)

{1: [[0.0017684648905237077, 4998.999999719896, 1.0001796093458002, 2466.2610280627628, 7375.69755241689], [1.0017684648905236, 5000.0, 1.0001796093458002, 2466.2610280627628, 7376.69755241689], [1.0017684648905236, 4999.999999719896, 1.0001796093458002, 2466.2610280627628, 7376.69755241689], [0.0, 4999.999999719896, 0.0, 2467.2610280627628, 7374.69755241689]], 2: [[0.0017684648905237077, 4998.999999719896, 0.00017960934580030057, 2466.2610280627628, 7374.69755241689], [0.0017684648905237077, 4999.999999719896, 0.0, 2465.2610280627628, 7376.69755241689], [1.0017684648905236, 4999.999999719896, 0.0, 2465.2610280627628, 7376.69755241689], [0.0017684648905237077, 4999.999999719896, 1.0001796093458002, 2465.2610280627628, 7374.69755241689]]}


c:\Users\Jacob\miniconda3\envs\calliope\lib\site-packages\deap\creator.py:185: RuntimeWarning:

A class named 'FitnessMaxDist' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.

c:\Users\Jacob\miniconda3\envs\calliope\lib\site-packages\deap\creator.py:185: RuntimeWarning:

A class named 'IndividualSP' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.



In [26]:
#calculate centroids and fitness
centroids = centroidSP(subpops_SP)
fitness_populations = fitness(subpops_SP, centroids)

# Combine the fitness values with each individual
for i, subpopulation in subpops_SP.items():
    for individual, fit in zip(subpopulation, fitness_populations[i]):
        individual.fitness.values = fit 

for subpop_index, subpopulation in subpops_SP.items():      
    # Calculate slack feasibility and set fitness accordingly. This is also where the cost gets assigned as an attribute to the individual
    for idx, individual in enumerate(subpopulation):  # Use enumerate to get the index
        slack_validity = slack_feasibility(individual)
        if slack_validity:
            individual.fitness.values = individual.fitness.values
        else:
            individual.fitness.values = (0,)
                
        # Print the required details in one line
        print(f"Feasibility: {slack_validity}, Fitness: {individual.fitness.values}, Cost: {individual.cost}, Values: {individual[:]}, Subpop: {subpop_index}, Ind: {idx + 1}")

Feasibility: True, Fitness: (0.0,), Cost: 28581150.5, Values: [0.0017684648905237077, 4998.999999719896, 1.0001796093458002, 2466.2610280627628, 7375.69755241689], Subpop: 1, Ind: 1
Feasibility: True, Fitness: (0.25000028010435926,), Cost: 28586631.05, Values: [1.0017684648905236, 5000.0, 1.0001796093458002, 2466.2610280627628, 7376.69755241689], Subpop: 1, Ind: 2
Feasibility: True, Fitness: (0.25,), Cost: 28586631.03, Values: [1.0017684648905236, 4999.999999719896, 1.0001796093458002, 2466.2610280627628, 7376.69755241689], Subpop: 1, Ind: 3
Feasibility: True, Fitness: (0.25,), Cost: 28586403.56, Values: [0.0, 4999.999999719896, 0.0, 2467.2610280627628, 7374.69755241689], Subpop: 1, Ind: 4
Feasibility: True, Fitness: (0.25,), Cost: 28580302.23, Values: [0.0017684648905237077, 4998.999999719896, 0.00017960934580030057, 2466.2610280627628, 7374.69755241689], Subpop: 2, Ind: 1
Feasibility: True, Fitness: (0.24999992997436493,), Cost: 28585236.439999998, Values: [0.0017684648905237077, 499

In [27]:
#unmet demand penalty. Must be changed when applied to a large model
unmetdemandvariable = 30000

In [28]:
# Initialize containers to store the fitness statistics
avg_fitness_per_gen = {i: [] for i in range(1, NUM_SUBPOPS + 1)}  # For average fitness (existing logic)
highest_fitness_per_gen = {i: [] for i in range(1, NUM_SUBPOPS + 1)}  # For highest fitness
highest_fitness_sum_per_gen = []  # For sum of highest fitness values across subpopulations

best_fitness_sum = float('-inf')  # Start with a very low value
best_individuals = []  # List to store the best individuals

low = [b[0] for b in low_up_bound]
up = [b[1] for b in low_up_bound]

elite_selections = {}

g = 0
while g < GENERATIONS: 
    g += 1
    print(f"-- Generation {g} --")

    offspring = {}
    current_individuals = []  # Track individuals contributing to this generation's highest sum
    highest_fitness_sum = 0  # Initialize the sum of highest fitness for this generation

    for subpop_index, subpopulation in subpops_SP.items():
        # Compute and store fitness values, excluding (0,) fitness values
        fitness_values = [ind.fitness.values[0] for ind in subpopulation if ind.fitness.values[0] != 0]

        # Calculate the average fitness (existing logic)
        if fitness_values:
            avg_fitness = sum(fitness_values) / len(fitness_values)
        else:
            avg_fitness = 0
        avg_fitness_per_gen[subpop_index].append(avg_fitness)

        # Calculate the highest fitness
        if fitness_values:
            highest_fitness = max(fitness_values)
        else:
            highest_fitness = 0
        highest_fitness_per_gen[subpop_index].append(highest_fitness)

        # Add to the total highest fitness sum for this generation
        highest_fitness_sum += highest_fitness

        # Identify the individual(s) contributing to the highest fitness
        best_individual = min(
            (ind for ind in subpopulation if ind.fitness.values[0] == highest_fitness),
            key=lambda ind: getattr(ind, 'cost', float('inf'))  # Select based on cost
        )

        # Add this individual to current_individuals
        current_individuals.append({
            "subpop_index": subpop_index,
            "fitness": best_individual.fitness.values[0],
            "cost": getattr(best_individual, 'cost', 0),
            "generation": g,  # Add the generation(best_individual, 'cost', 0),
            "values": list(best_individual)
        })

        # Select the next generation individuals
        # Preserve the top ~% as elites and select the rest through tournament selection
        elite_count = int(0.2 * len(subpopulation))
        elite_selections[subpop_index] = toolbox.elitism(subpopulation, elite_count)
        offspring[subpop_index] = (elite_selections[subpop_index] + toolbox.tournament(subpopulation, (len(subpopulation) - elite_count)))

        # Clone the selected individuals
        offspring[subpop_index] = list(map(toolbox.clone, offspring[subpop_index]))


        # Apply crossover
        for child1, child2 in zip(offspring[subpop_index][::2], offspring[subpop_index][1::2]): 
            if random.random() < 0.5:  # Use updated crossover probability
                toolbox.mate(child1, child2, indpb=0.5) 
                del child1.fitness.values 
                del child2.fitness.values
                del child1.cost
                del child2.cost 

        # Apply mutation
        for mutant in offspring[subpop_index]:
            if random.random() <= 1:
                # Apply mutPolynomialBounded with shared bounds
                mutant, = toolbox.mutbound(mutant, low=low, up=up, eta=ETAV, indpb=INDMUT)
                mutant[:] = [max(0, val) for val in mutant]  # Ensure values are non-negative
                
                # Delete fitness to ensure re-evaluation
                if hasattr(mutant.fitness, 'values'):
                    del mutant.fitness.values
                
                if hasattr(mutant.cost, 'values'):
                    del mutant.cost

    

    # Append the total highest fitness sum for this generation
    highest_fitness_sum_per_gen.append(highest_fitness_sum)

    # Check if the current highest fitness sum is the best we've seen
    if highest_fitness_sum > best_fitness_sum:
        best_fitness_sum = highest_fitness_sum
        best_individuals = current_individuals



    # Calculate slack feasibility and set fitness accordingly
    feasible_individuals = {subpop_index: [] for subpop_index in offspring.keys()}  # Dictionary format
    infeasible_individuals = {subpop_index: [] for subpop_index in offspring.keys()}

    for subpop_index, subpopulation in offspring.items():
        # Step 1: Calculate slack feasibility
        for idx, individual in enumerate(subpopulation):
            slack_validity = slack_feasibility(individual)

            if slack_validity:
                feasible_individuals[subpop_index].append(individual)
                print(f"Feasible - Subpop: {subpop_index}, Ind: {idx + 1}, Values: {individual}, Fitness: {individual.fitness.values}")
            else:
                # Replace infeasible individuals with elites
                if elite_selections[subpop_index]:  # Ensure there are elites left for this subpopulation
                    replacement = elite_selections[subpop_index].pop(0)  # Take one elite from this subpopulation's selection
                    subpopulation[idx] = toolbox.clone(replacement)  # Replace with a clone of the elite
                    feasible_individuals[subpop_index].append(subpopulation[idx])  # Add to feasible
                    print(f"Replaced with Elite - Subpop: {subpop_index}, Ind: {idx + 1}, Values: {subpopulation[idx]}, Fitness: {subpopulation[idx].fitness.values}")
                else:
                    # If no elites are left, assign zero fitness
                    individual.fitness.values = (0,)
                    infeasible_individuals[subpop_index].append(individual)
                    print(f"Infeasible - Subpop: {subpop_index}, Ind: {idx + 1}, Values: {individual}, Fitness: {individual.fitness.values}")


        
    # Step 2: Calculate centroids and fitness for feasible individuals
    if feasible_individuals:  # Ensure there are feasible individuals
        centroids_offspring = copy.deepcopy(centroidSP(feasible_individuals))
        fitness_SP_offspring = fitness(feasible_individuals, centroids_offspring)

        # Debug print: Check centroids and fitness before assignment
        print("Centroids for Offspring:", centroids_offspring)
        print("Fitness Calculated for Feasible Individuals:", fitness_SP_offspring)

        # Assign calculated fitness to feasible individuals
        for subpop_index, individuals in feasible_individuals.items():
            print(f"Assigning fitness to Subpopulation {subpop_index}:")  # Debug print
            if individuals:  # Ensure there are individuals to process
                for idx, individual in enumerate(individuals):
                    print(f"Before Assignment - Ind: {idx + 1}, Fitness: {individual.fitness.values}")
                    individual.fitness.values = fitness_SP_offspring[subpop_index][idx]
                    print(f"After Assignment - Ind: {idx + 1}, Fitness: {individual.fitness.values}")
            else:
                print(f"Warning: No feasible individuals in Subpopulation {subpop_index}")


    # Combine feasible and infeasible individuals to form the new offspring
    for subpop_index in offspring.keys():
        print(f"--- Debugging Subpopulation {subpop_index} ---")
        
        # Print counts of feasible and infeasible individuals
        feasible_count = len(feasible_individuals[subpop_index])
        infeasible_count = len(infeasible_individuals[subpop_index])
        print(f"Feasible Count: {feasible_count}, Infeasible Count: {infeasible_count}")

        # Validate total population size
        original_size = len(offspring[subpop_index])
        combined_size = feasible_count + infeasible_count
        print(f"Original Population Size: {original_size}, Combined Size After Merge: {combined_size}")
        assert combined_size == original_size, (
            f"Mismatch in population size for Subpopulation {subpop_index}: "
            f"Original: {original_size}, Combined: {combined_size}"
        )
        
        # Print a few individuals for validation
        print("Feasible Individuals:")
        for idx, ind in enumerate(feasible_individuals[subpop_index][:5], start=1):  # Print up to 5 feasible individuals
            print(f"  Ind {idx}: Fitness: {ind.fitness.values}, Values: {ind}")
        
        print("Infeasible Individuals:")
        for idx, ind in enumerate(infeasible_individuals[subpop_index][:5], start=1):  # Print up to 5 infeasible individuals
            print(f"  Ind {idx}: Fitness: {ind.fitness.values}, Values: {ind}")
        
        # Combine and update offspring
        offspring[subpop_index] = feasible_individuals[subpop_index] + infeasible_individuals[subpop_index]
        print(f"New Offspring Size: {len(offspring[subpop_index])}")
        print("-" * 40)

    # Step 3: Print generation summary for all subpopulations
    for subpop_index, subpopulation in offspring.items():
        for idx, individual in enumerate(subpopulation):
            cost = getattr(individual, 'cost', 'N/A')  # Safeguard for missing cost attribute
            print(f"Fitness: {individual.fitness.values}, Cost: {cost}, Values: {individual}, Subpop: {subpop_index}, Ind: {idx + 1}")
    print("-" * 40)

    #assign offspring to subpops_SP so that created generation goes to the next loop
    subpops_SP = offspring


-- Generation 1 --
Infeasible - Subpop: 1, Ind: 1, Values: [1.0017684648905236, 4999.999999719896, 0.06433986317407081, 4054.85821327505, 7376.69755241689], Fitness: (0.0,)
Infeasible - Subpop: 1, Ind: 2, Values: [342.62580837611966, 4233.881353127345, 1.0001796093458002, 2466.2610280627628, 5618.961004803648], Fitness: (0.0,)
Infeasible - Subpop: 1, Ind: 3, Values: [0.0014494421191004747, 5000.0, 1.0001796093458002, 2819.3771067380358, 9603.349260289619], Fitness: (0.0,)
Infeasible - Subpop: 1, Ind: 4, Values: [1.0017684648905236, 4983.414569569227, 1.0001796093458002, 2466.2610280627628, 7376.69755241689], Fitness: (0.0,)
Infeasible - Subpop: 2, Ind: 1, Values: [1.0017684648905236, 4999.999999719896, 0, 2465.2610280627628, 7376.69755241689], Fitness: (0.0,)
Infeasible - Subpop: 2, Ind: 2, Values: [0.0017684648905237077, 4999.999999770595, 0, 2465.2610280627628, 4070.428450426775], Fitness: (0.0,)
Infeasible - Subpop: 2, Ind: 3, Values: [795.3397444128401, 4998.999999719896, 0.0001010

IndexError: list index out of range

In [13]:
# Export Best Individuals to Excel after the loop finishes
filename = "individual_generation_interval.xlsx"
try:
    with pd.ExcelWriter(filename, mode='a', engine='openpyxl') as writer:
        best_individuals_df.to_excel(writer, sheet_name="Best_Individuals", index=False)
except FileNotFoundError:
    with pd.ExcelWriter(filename, mode='w', engine='openpyxl') as writer:
        best_individuals_df.to_excel(writer, sheet_name="Best_Individuals", index=False)

In [ ]:
print("\nBest Individuals Across All Generations:")
print(f"Highest Fitness Sum: {best_fitness_sum}")
for ind in best_individuals:
    print(f"  Generation {ind['generation']} - Subpopulation {ind['subpop_index']} - "
        f"Fitness: {ind['fitness']:.2f}, Cost: {ind['cost']:.2f}, Values: {ind['values']}")